# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8544f58be0>
├── 'a' --> tensor([[-0.3138,  0.1918, -0.8106],
│                   [ 0.4303,  0.5945, -1.4270]])
└── 'x' --> <FastTreeValue 0x7f8544f584f0>
    └── 'c' --> tensor([[-0.2312, -1.0278,  0.8283, -1.4376],
                        [ 0.2737,  0.2202,  0.0034, -0.3519],
                        [ 1.2404,  0.9185,  1.9489,  0.0638]])

In [4]:
t.a

tensor([[-0.3138,  0.1918, -0.8106],
        [ 0.4303,  0.5945, -1.4270]])

In [5]:
%timeit t.a

66.6 ns ± 0.115 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8544f58be0>
├── 'a' --> tensor([[ 0.0270,  0.7709, -1.4366],
│                   [ 0.4031, -0.6681, -0.0788]])
└── 'x' --> <FastTreeValue 0x7f8544f584f0>
    └── 'c' --> tensor([[-0.2312, -1.0278,  0.8283, -1.4376],
                        [ 0.2737,  0.2202,  0.0034, -0.3519],
                        [ 1.2404,  0.9185,  1.9489,  0.0638]])

In [7]:
%timeit t.a = new_value

72.4 ns ± 0.115 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3138,  0.1918, -0.8106],
               [ 0.4303,  0.5945, -1.4270]]),
    x: Batch(
           c: tensor([[-0.2312, -1.0278,  0.8283, -1.4376],
                      [ 0.2737,  0.2202,  0.0034, -0.3519],
                      [ 1.2404,  0.9185,  1.9489,  0.0638]]),
       ),
)

In [10]:
b.a

tensor([[-0.3138,  0.1918, -0.8106],
        [ 0.4303,  0.5945, -1.4270]])

In [11]:
%timeit b.a

57.7 ns ± 0.127 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7384,  1.7014,  1.7633],
               [-1.0095,  0.5973,  1.9927]]),
    x: Batch(
           c: tensor([[-0.2312, -1.0278,  0.8283, -1.4376],
                      [ 0.2737,  0.2202,  0.0034, -0.3519],
                      [ 1.2404,  0.9185,  1.9489,  0.0638]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.51 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

826 ns ± 0.491 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 18.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 633 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f848e210040>
├── 'a' --> tensor([[[-0.3138,  0.1918, -0.8106],
│                    [ 0.4303,  0.5945, -1.4270]],
│           
│                   [[-0.3138,  0.1918, -0.8106],
│                    [ 0.4303,  0.5945, -1.4270]],
│           
│                   [[-0.3138,  0.1918, -0.8106],
│                    [ 0.4303,  0.5945, -1.4270]],
│           
│                   [[-0.3138,  0.1918, -0.8106],
│                    [ 0.4303,  0.5945, -1.4270]],
│           
│                   [[-0.3138,  0.1918, -0.8106],
│                    [ 0.4303,  0.5945, -1.4270]],
│           
│                   [[-0.3138,  0.1918, -0.8106],
│                    [ 0.4303,  0.5945, -1.4270]],
│           
│                   [[-0.3138,  0.1918, -0.8106],
│                    [ 0.4303,  0.5945, -1.4270]],
│           
│                   [[-0.3138,  0.1918, -0.8106],
│                    [ 0.4303,  0.5945, -1.4270]]])
└── 'x' --> <FastTreeValue 0x7f8544fbadc0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.2 µs ± 86.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f848e2101f0>
├── 'a' --> tensor([[-0.3138,  0.1918, -0.8106],
│                   [ 0.4303,  0.5945, -1.4270],
│                   [-0.3138,  0.1918, -0.8106],
│                   [ 0.4303,  0.5945, -1.4270],
│                   [-0.3138,  0.1918, -0.8106],
│                   [ 0.4303,  0.5945, -1.4270],
│                   [-0.3138,  0.1918, -0.8106],
│                   [ 0.4303,  0.5945, -1.4270],
│                   [-0.3138,  0.1918, -0.8106],
│                   [ 0.4303,  0.5945, -1.4270],
│                   [-0.3138,  0.1918, -0.8106],
│                   [ 0.4303,  0.5945, -1.4270],
│                   [-0.3138,  0.1918, -0.8106],
│                   [ 0.4303,  0.5945, -1.4270],
│                   [-0.3138,  0.1918, -0.8106],
│                   [ 0.4303,  0.5945, -1.4270]])
└── 'x' --> <FastTreeValue 0x7f848e210070>
    └── 'c' --> tensor([[-0.2312, -1.0278,  0.8283, -1.4376],
                        [ 0.2737,  0.2202,  0.0034, -0.3519],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 61.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.6 µs ± 53.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.2312, -1.0278,  0.8283, -1.4376],
                       [ 0.2737,  0.2202,  0.0034, -0.3519],
                       [ 1.2404,  0.9185,  1.9489,  0.0638]],
              
                      [[-0.2312, -1.0278,  0.8283, -1.4376],
                       [ 0.2737,  0.2202,  0.0034, -0.3519],
                       [ 1.2404,  0.9185,  1.9489,  0.0638]],
              
                      [[-0.2312, -1.0278,  0.8283, -1.4376],
                       [ 0.2737,  0.2202,  0.0034, -0.3519],
                       [ 1.2404,  0.9185,  1.9489,  0.0638]],
              
                      [[-0.2312, -1.0278,  0.8283, -1.4376],
                       [ 0.2737,  0.2202,  0.0034, -0.3519],
                       [ 1.2404,  0.9185,  1.9489,  0.0638]],
              
                      [[-0.2312, -1.0278,  0.8283, -1.4376],
                       [ 0.2737,  0.2202,  0.0034, -0.3519],
                       [ 1.2404,  0.9185,  1.9489,  0.0638]],

In [26]:
%timeit Batch.stack(batches)

76.8 µs ± 88.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.2312, -1.0278,  0.8283, -1.4376],
                      [ 0.2737,  0.2202,  0.0034, -0.3519],
                      [ 1.2404,  0.9185,  1.9489,  0.0638],
                      [-0.2312, -1.0278,  0.8283, -1.4376],
                      [ 0.2737,  0.2202,  0.0034, -0.3519],
                      [ 1.2404,  0.9185,  1.9489,  0.0638],
                      [-0.2312, -1.0278,  0.8283, -1.4376],
                      [ 0.2737,  0.2202,  0.0034, -0.3519],
                      [ 1.2404,  0.9185,  1.9489,  0.0638],
                      [-0.2312, -1.0278,  0.8283, -1.4376],
                      [ 0.2737,  0.2202,  0.0034, -0.3519],
                      [ 1.2404,  0.9185,  1.9489,  0.0638],
                      [-0.2312, -1.0278,  0.8283, -1.4376],
                      [ 0.2737,  0.2202,  0.0034, -0.3519],
                      [ 1.2404,  0.9185,  1.9489,  0.0638],
                      [-0.2312, -1.0278,  0.8283, -1.4376],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 204 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

327 µs ± 2.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
